<a href="https://colab.research.google.com/github/23Aditya/TransferLearning-in-Keras/blob/main/offline_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.applications import vgg16
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense,Flatten,BatchNormalization,InputLayer
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np

In [ ]:
data_gen = ImageDataGenerator()

In [ ]:
training_datagen = data_gen.flow_from_directory(directory="/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/Train",
                                                     target_size=(32,32),color_mode="rgb",
                                                     classes=os.listdir("/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/Train"),
                                                     batch_size=50)

Found 39440 images belonging to 29 classes.


In [ ]:
cv_datagen = data_gen.flow_from_directory(directory="/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/cv",
                                                    target_size=(32,32),color_mode="rgb",
                                                    classes=os.listdir("/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/cv"),
                                                    batch_size=15640)

Found 0 images belonging to 46 classes.


In [ ]:
testing_datagen = data_gen.flow_from_directory(directory="/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/Test",
                                                   target_size=(32,32),color_mode="rgb",
                                                   classes=os.listdir("/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/Test"),
                                                   batch_size=13800)

Found 13800 images belonging to 46 classes.


In [ ]:
conv_base = vgg16.VGG16(include_top=False,input_shape=(32,32,3))

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
conv_base.trainable = False

In [ ]:
base_dir = "/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset"

In [ ]:
training_dir = os.path.join(base_dir,"Train")
cv_dir = os.path.join(base_dir,"cv")
testing_dir = os.path.join(base_dir,"Test")

In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [ ]:
def fetch_conv_base_output(train_cv_test_dir,train_cv_test_size):
  conv_base_output = np.zeros(shape=(train_cv_test_size,1,1,512))
  train_cv_test_labels = np.zeros(shape=(train_cv_test_size,46))

  train_cv_test_datagen = data_gen.flow_from_directory(directory=train_cv_test_dir,
                                                       target_size=(32,32),color_mode="rgb",
                                                       classes=os.listdir(train_cv_test_dir),batch_size=40)
  i=0
  batch_size = 40

  for input_image_batch,labels_batch in train_cv_test_datagen:

    conv_base_output_batch = conv_base.predict(input_image_batch)
    conv_base_output[i*batch_size:(i+1)*batch_size] = conv_base_output_batch
    train_cv_test_labels[i*batch_size:(i+1)*batch_size] = labels_batch
    i = i + 1

    print("Processed Batch Number",1)

    if i*batch_size >  train_cv_test_size:
      break

  return conv_base_output,train_cv_test_labels



In [ ]:
conv_base_test_output = fetch_conv_base_output(testing_dir,13800)

Found 13800 images belonging to 46 classes.
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Processed Batch Number 1
Proces

ValueError: ignored

In [ ]:
def build_network():

  network = Sequential()
  network.add(InputLayer(input_shape=(1,1,512)))
  network.add(Dense(units=512,activation="relu"))
  network.add(Dense(units=46,activation="softmax"))

  network.compile(optimizer=Adam(),loss="categorical_crossentropy",metrics=["accuracy"])

  return network

In [ ]:
conv_base_test_output_imgs = conv_base_test_output
test_labels = conv_base_test_output[1]

NameError: ignored

In [ ]:
images_tensor_file_handle = open("/content/drive/MyDrive/Colab Notebooks/conv_base_output_imgs.npz",mode="w")

labels_file_handle = open("/content/drive/MyDrive/Colab Notebooks/test_labels.npz",mode="w")


In [ ]:
np.savez("/content/drive/MyDrive/Colab Notebooks/conv_base_output_imgs.npz",conv_base_output_imgs)
np.savez("/content/drive/MyDrive/Colab Notebooks/test_labels.npz",test_labels)

In [ ]:
test_datagen = datagen.flow(x=conv_base_test_output_imgs,y=test_labels,batch_size=40)

In [ ]:
network = build_network()

In [ ]:
network.summary()

In [ ]:
network.fit_generator(generator=test_datagen,epochs=1,workers=2,use_multiprocessing=True)